# **Kinetic model environment**

The `kinetic model` environment allows us to simulate microbial communities and their environments using ordinary differential equations. To get a better understanding of the model, 
We are going to create a simplified example of *E. coli* subpopulations. In our example, there are three subpopulations, two active and one inactive. The active ones feed respectively on glucose releasing acetate (overflow) and on acetate releasing CO2. Both subpopulations can become inactive.


<div style="height:24px;"></div>

```css
Glucose ---> [Subpopulation A] ---> Acetate
Acetate ---> [Subpopulation B] ---> CO2

[Subpopulation A] ---> [Inactive]
[Subpopulation B] ---> [Inactive]
```

<div style="height:24px;"></div>


#### **Metabolites & Metabolome**

We first create three Metabolite objects for glucose, acetate, and CO2 and group them in a Metabolome.


In [ ]:
#import packages for the model and visualization
from kinetic_model.metabolite import Metabolite
from kinetic_model.visualize import GraphSpecBuilder, CytoscapeExporter
import json


#create metabolites
glucose = Metabolite(name = "glucose", concentration = 5.0, formula ={'C': 6, 'H': 12, 'O': 6}, color = '#ff0000')

